In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import surp
from surp import gce_math as gcem

In [ ]:
import vice

- P16: 10.26093/cds/vizier.22250024. Table 12. 
- R18 is available at DOI: 10.11570/18.0002

In [ ]:
path = "/astro/dboyea/source/VICE/vice/yields/agb/pignatari16/raw/"
datadir = "../../data/"

## P16

Pignatari et al. (2016) reports yields in terms of over production factors (OP), defined as
$$
{\rm OP} = \frac{M_{\rm ej, i}}{M_{\rm ej} Z_{i}^0}
$$
 i.e. the ratio between the mean ejected abundance composition and the initial composition of the star. 
 Since we are interested in net fractional yields, we can calculate these from the overproduction factor as follows
 $$
 Y_{X} = Z_{\rm ini} ({\rm OP} - 1) \frac{M_{\rm ej}}{M_{\rm ini}} = (Z_{\rm ej} - Z_{\rm ini}) \frac{M_{\rm ej}}{M_{\rm ini}}
 $$.

Or 
$$
OP = 1 + \frac{Y_X}{Z_{\rm ini}} \frac{M_{\rm ini}}{M_{\rm ej}}
$$
 
Thus from each nu-grid model, we need the OP factor, the initial and ejected mass, and the birth composition.

In [ ]:
Zsun = pd.read_csv(datadir + "grevese_noels_93.txt", comment="#", names=["element", "epsilon"])

Zsun["Z"] = gcem.eps_to_abundance(Zsun["epsilon"], Zsun["element"].values)
print(sum(Zsun.Z))
Zsun["Z"] /= sum(Zsun.Z)
Zsun.set_index("element", inplace=True)
Zsun

In [ ]:
P16_stars = pd.read_csv(datadir + "pignatari16_stars.tsv", sep="\s+", comment="#")
P16_stars

In [ ]:
P16_stars.set_index(["mass", "Z"], inplace=True)
P16_stars

In [ ]:
P16_ops_raw = pd.read_csv(datadir + "pignatari16_table12.tsv", sep="\t", comment="#", skiprows=[42, 43])
P16_ops_raw

For P16, note that set 1.1 has Z=0.01, set 1.2 has Z=0.02.

In [ ]:
P16_ops_raw["Z"] = np.where(P16_ops_raw.Set == 1.2, 0.02, 0.01)

In [ ]:
P16_ops = P16_ops_raw.melt(id_vars=["Sp", "Z"], value_vars=["OP1.65", "OP2", "OP3", "OP4", "OP5",], var_name="model", value_name="OP")
P16_ops

In [ ]:
P16_ops["M"] = P16_ops.model.str.extract(r"(\d?\.?\d+)")
P16_ops["M"] = pd.to_numeric(P16_ops.M)
P16_ops

In [ ]:
P16 = P16_ops

In [ ]:
m_rem = [P16_stars.loc[r.M, r.Z][0] for i, r in P16.iterrows()]

In [ ]:
P16["Mrem"] = m_rem

In [ ]:
P16["element"] = P16.Sp.str.strip()

In [ ]:
Z0 = [Zsun.loc[r.element].Z for i, r in P16.iterrows()]

In [ ]:
P16["Zini"] = Z0 * P16.Z / 0.0179

In [ ]:
P16

In [ ]:
P16["mass_yield"] = P16.Zini * (P16.OP - 1) * (1 - P16.Mrem/P16.M)

In [ ]:
plt.scatter(P16.M, P16.Mrem)
plt.xlabel("M ini / msun")
plt.ylabel("reminant mass / msun")

## R 18

In [ ]:
elements = "H, He, Li, B, C, N, O, F, Ne, Na, Mg, Al, Si, P, Pb, S, Cl, Ar, K, Ca, Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Y, Zr, Nb, Mo, Ru, Rh, Pd, Ag, Cd, In, Sn, Sb, Te, I, Xe, Cs, Ba, La, Ce, Pr, Nd, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg, Tl, Bi".split(", ")

In [ ]:
R18_raw = pd.DataFrame()


with open(path + "/element_yield_table_MESAonly_fryer12_delay_total.txt") as file:
    skip = 6
    M = None
    M_ej = None
    for line in file:
        if skip > 0:
            skip -= 1
            continue
            
        if line[:7] == "H Table":
            s1, s2 = line[10:].split(",")
            M = float(s1[2:])
            Z = float(s2[2:-2])
            skip = 1
            continue
            
        if line[:8] == "H Mfinal":
            M_rem = float(line[10:])
            M_ej = M - M_rem
            skip = 1
            continue
            
        if M_ej is None:
            continue
        
        _, ele, y, x0, _ = line.split("&")

        y = float(y)
        x0 = float(x0)

        row = pd.DataFrame({"M":M, "Z":Z, "element":ele.strip(), "Mrem":M_rem, "mass_yield":y, "Zini": x0}, index=[0])
        R18_raw = pd.concat([R18_raw, row], ignore_index=True)
        


In [ ]:
R18 = R18_raw
R18["OP"] = R18.mass_yield / ((R18.M -R18.Mrem) * R18.Zini)


In [ ]:
R18.M

In [ ]:
R18_indexed = R18_raw.set_index(["M", "Z", "element"])

In [ ]:
R18_indexed.sort_index(inplace=True)

In [ ]:
R18_indexed.index

## B19/21

In [ ]:
b19_raw2 = pd.read_csv(path + "/B19.txt", sep="\s+", skiprows=0, index_col=0)
b21_raw2 = pd.read_csv(path + "/B21.txt", sep="\s+", skiprows=2, index_col=0)

In [ ]:
b19_raw = pd.read_csv(path + "/B19_2.txt", sep="\s+", skiprows=2, index_col=0)
b21_raw = pd.read_csv(path + "/B21_2.txt", sep="\s+", skiprows=2, index_col=0)

In [ ]:
np.sum(b19_raw[b19_raw.index.str.startswith("K-")], axis=0).iloc[0]

In [ ]:
R18_indexed.loc[2., 0.01, "K"].Zini.values

In [ ]:
B19_indexed.loc[2., 0.01, "K"].mass_yield.values

In [ ]:
Zsun

In [ ]:
for ele in elements[4:]:
    Zr = R18_indexed.loc[1., 0.01, ele].Zini.values[0] * Z0 / 0.01
    Zgn = Zsun.Z[ele]
    print(f"{ele}\t{vice.solar_z(ele):10.3e}\t{Zr:10.3e}\t{Zgn:10.3e}")
    

In [ ]:
np.std(b19_raw2 / b19_raw), np.mean(b19_raw / b19_raw2)

In [ ]:
np.std(b21_raw2 / b21_raw)

In [ ]:
b19_ele = pd.DataFrame()
for idx, row in b19_raw.iterrows():
    ele, _ = idx.split("-")
    if ele in b19_ele.index:
        b19_ele.loc[ele] += row
    else:
        b19_ele = pd.concat([b19_ele, pd.DataFrame(row.to_dict(), index=[ele])])

In [ ]:
b21_ele = pd.DataFrame()
for idx, row in b21_raw.iterrows():
    ele, _ = idx.split("-")
    if ele in b21_ele.index:
        b21_ele.loc[ele] += row
    else:
        b21_ele = pd.concat([b21_ele, pd.DataFrame(row.to_dict(), index=[ele])])

In [ ]:
b21_ele["element"] = b21_ele.index
b19_ele["element"] = b19_ele.index

b21_ele

In [ ]:
B19 = b19_ele.melt(id_vars=["element"],  var_name="model", value_name="mass_yield")

B19["Z"] = 0.
B19["M"] = 0.
B19

B19.loc[B19.model == "m2z1m2", "M"] = 2.
B19.loc[B19.model == "m2z1m2", "Z"] = 0.01
B19.loc[B19.model == "m2z1m2", "Mrem"] = 0.632

B19.loc[B19.model == "m3z1m2", "M"] = 3.
B19.loc[B19.model == "m3z1m2", "Z"] = 0.01
B19.loc[B19.model == "m3z1m2", "Mrem"] = 0.661


B19.loc[B19.model == "m2z2m2", "M"] = 2.
B19.loc[B19.model == "m2z2m2", "Z"] = 0.02
B19.loc[B19.model == "m2z2m2", "Mrem"] = 0.646


B19.loc[B19.model == "m3z2m2", "M"] = 3.
B19.loc[B19.model == "m3z2m2", "Z"] = 0.02
B19.loc[B19.model == "m3z2m2", "Mrem"] = 0.656	


B19[B19.element == "K"]

In [ ]:



B19 = B19[np.isin(B19.element, elements)]
B19 = B19[B19.M > 0]


B19["Zini"] = [Zsun.Z[r.element] * r.Z / Z0 for (i, r) in B19.iterrows()]

B19["OP"] = B19.mass_yield / ((B19.M - B19.Mrem) * B19.Zini)
B19_indexed = B19.set_index(["M", "Z", "element"])


B19[B19.element == "K"]


In [ ]:
B19.Mrem.unique()

In [ ]:
B21 = b21_ele.melt(id_vars=["element"],  var_name="model", value_name="mass_yield")

B21["M"] = 0.
B21["Z"] = 0.

B21.loc[B21.model == "m3z1m3-bigpoc", "M"] = 3.
B21.loc[B21.model == "m3z1m3-bigpoc", "Z"] = 0.001

# B21.loc[B21.model == "m2z2m3-bigpoc", "M"] = 2.
# B21.loc[B21.model == "m2z2m3-bigpoc", "Z"] = 0.002

B21.loc[B21.model == "m2z1m3-bigpoc", "M"] = 2.
B21.loc[B21.model == "m2z1m3-bigpoc", "Z"] = 0.001


B21 = B21[B21.M > 0]
B21 = B21[np.isin(B21.element, elements)]

#TODO technically incorrect
B21["Mrem"] = [R18_indexed.loc[r.M, r.Z, r.element].Mrem.values[0] for (i, r) in B21.iterrows()]
B21["Zini"] = [R18_indexed.loc[r.M, r.Z, r.element].Zini.values[0] for (i, r) in B21.iterrows()]
B21["OP"] = B21.mass_yield / ((B21.M - B21.Mrem) * B21.Zini)




B21

In [ ]:
R18_indexed.loc[1.0, 0.01, "C"].Mrem.values[0]

In [ ]:
B21_indexed = B21.set_index(["M", "Z", "element"])

In [ ]:
R18_indexed.Mrem[2., 0.01, "H"].values[0]

In [ ]:
B19_indexed.Mrem[2., 0.01, "C"]

In [ ]:
P16

In [ ]:
R18

# Comparisons

In [ ]:
def mrem(m):
    return 0.394 + 0.109 * m

In [ ]:
Z0 = np.sum(Zsun.Z[2:])

In [ ]:
Z0

In [ ]:
Zsun

In [ ]:
def vice_yield_to_mass_yield_gn(M, z, ele, study="cristallo11"):
    y, ms, zs = vice.yields.agb.grid(ele, study=study)

    i = np.where(M == np.array(ms))[0][0]
    j = np.where(z == np.array(zs))[0][0]

    Mrem = R18_indexed.loc[M, z, ele].Mrem.values[0]
    Zini = R18_indexed.loc[M, z, ele].Zini.values[0]
    
    Mej = M - Mrem
    nfy = y[i][j]
    net_yield = M * nfy
    yld  = net_yield +   Mej* Zini
    return float(yld)
                             

In [ ]:
def vice_yield(M, z, ele, study):
    y, ms, zs = vice.yields.agb.grid(ele, study=study)

    i = np.where(M == np.array(ms))[0][0]
    j = np.where(z == np.array(zs))[0][0]

    return y[i][j]

In [ ]:
def vice_yield_to_mass_yield(M, z, ele, study="cristallo11"):
    y, ms, zs = vice.yields.agb.grid(ele, study=study)

    i = np.where(M == np.array(ms))[0][0]
    j = np.where(z == np.array(zs))[0][0]

    Mej = M - mrem(M)
    nfy = y[i][j]
    net_yield = M * nfy
    Zini = vice.solar_z(ele) * z / 0.016
    yld  = net_yield + Mej * Zini
    return float(yld)
                             

In [ ]:
P16_indexed = P16.set_index(["M", "Z", "element"])

In [ ]:
print("ele\tP+16\t\tR+18\t\tB+19\t\tBPR\t\tC11\t\tK10")
for ele in ["C", "N", "O", "F", "Ne", "Mg", "Al", "P", "Pb"]:
    M = 2.0
    Z = 0.02
    print(f"{ele}", end="\t")

    y = P16_indexed.loc[M, Z, ele].mass_yield
    y = y
    print(f"{y:6.2e}", end="\t")

    
    y = R18_indexed.loc[M, Z, ele].mass_yield
    y = y.values[0]
    print(f"{y:6.2e}", end="\t")
    
    y = B19_indexed.loc[M, Z, ele].mass_yield
    print(f"{y:6.2e}", end="\t")



    for study in ["pignatari16", "cristallo11", "karakas10"]:
        try:
            y = vice_yield_to_mass_yield(M, Z, ele, study)
        except LookupError:
            y = np.nan
        print(f"{y:6.2e}", end="\t")


    print()

In [ ]:
print("ele\tP+16\t\tR+18\t\tB+19\t\tBPR\t\tC11\t\tK10")
for ele in ["C", "N", "O", "F", "Ne", "Mg", "Al", "P", "Pb"]:
    M = 3.0
    Z = 0.01
    print(f"{ele}", end="\t")

    y = P16_indexed.loc[M, Z, ele].mass_yield
    y = y
    print(f"{y:6.2e}", end="\t")

    
    y = R18_indexed.loc[M, Z, ele].mass_yield
    y = y.values[0]
    print(f"{y:6.2e}", end="\t")
    
    y = B19_indexed.loc[M, Z, ele].mass_yield
    print(f"{y:6.2e}", end="\t")



    for study in ["pignatari16", "cristallo11", "karakas10"]:
        try:
            y = vice_yield_to_mass_yield_gn(M, Z, ele, study)
        except LookupError:
            y = np.nan
        print(f"{y:6.2e}", end="\t")


    print()

In [ ]:
P16.Z.unique()

In [ ]:
print("ele\tP+16\t\tR+18\t\tBPR\t\t\C11\t\tK10")
for ele in ["C", "N", "O", "F", "Ne", "Mg", "Al", "P", "Pb"]:
    M = 5.0
    Z = 0.02
    print(f"{ele}", end="\t")

    y = P16_indexed.loc[M, Z, ele].mass_yield
    y = y
    print(f"{y:6.2e}", end="\t")

    
    y = R18_indexed.loc[M, Z, ele].mass_yield
    y = y.values[0]
    print(f"{y:6.2e}", end="\t")


    for study in ["pignatari16", "cristallo11", "karakas10"]:
        try:
            y = vice_yield_to_mass_yield_gn(M, Z, ele, study)
        except LookupError:
            y = np.nan
        print(f"{y:6.2e}", end="\t")


    print()

In [ ]:
import molmass

In [ ]:
@np.vectorize
def atomic_numer(ele):
    return molmass.ELEMENTS[ele].number

In [ ]:
R18.M.unique()

In [ ]:
for M in [2., 3.]:
    for Z in [0.01, 0.02]:

        y = np.array([vice_yield(M, Z, ele, "pignatari16") for ele in elements[4:]])
        Z0 = R18_indexed.loc[M, Z, elements[4:]].Zini
        Mej = M - B19_indexed.loc[M, Z, "H"].Mrem

        op = 1 + y / Z0 * M / Mej 

        

        plt.scatter(atomic_numer(elements[4:]), op)
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )
        
        plt.legend()
        plt.title(f"M={M}, Z={Z}")
        plt.yscale("log")
    plt.show()

In [ ]:
for Z in [0.01, 0.02]:

    for M in [1.65, 2., 3., 5.]:
    
        x  = atomic_numer(P16_indexed.loc[M, Z, :].index.values)
        y = P16_indexed.loc[M, Z, :].OP
        plt.scatter(x, y, label="P+16")
        
        x  = atomic_numer(R18_indexed.loc[M, Z, :].index.values)
        y = R18_indexed.loc[M, Z, :].OP
        plt.scatter(x, y, label="R+18", s=3)
    
        if M in [2., 3.]:
            print("B19")
            print("Z=", Z)
            print("M=", M)
            print("Mrem", B19_indexed.Mrem[M, Z, "H"])
            print("model ", B19_indexed.model[M, Z, "H"])
            df = B19_indexed.Zini[M, Z, :]
            Z1 = np.sum(df[~np.isin(df.index, ["H", "He"])])
            print(f"Z = {Z1:0.4f}")
            
            x  = atomic_numer(B19_indexed.loc[M, Z, :].index.values)
            y = B19_indexed.loc[M, Z, :].OP
            plt.scatter(x, y, label="B+19", s=1)       
        
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )
        
        plt.legend()
        plt.title(f"M={M}, Z={Z}")
        plt.yscale("log")
        plt.show()

In [ ]:
for Z in [0.01, 0.02]:

    for M in [1.65, 2., 3., 5.]:
    
        x  = atomic_numer(P16_indexed.loc[M, Z, :].index.values)
        y = P16_indexed.loc[M, Z, :].OP
        plt.scatter(x, y, label="P+16")
        
        x  = atomic_numer(R18_indexed.loc[M, Z, :].index.values)
        y = R18_indexed.loc[M, Z, :].OP
        plt.scatter(x, y, label="R+18", s=3)
    
        if M in [2., 3.]:
            print("B19")
            print("Z=", Z)
            print("M=", M)
            print("Mrem", B19_indexed.Mrem[M, Z, "H"])
            print("model ", B19_indexed.model[M, Z, "H"])
            df = B19_indexed.Zini[M, Z, :]
            Z1 = np.sum(df[~np.isin(df.index, ["H", "He"])])
            print(f"Z = {Z1:0.4f}")
            
            x  = atomic_numer(B19_indexed.loc[M, Z, :].index.values)
            y = B19_indexed.loc[M, Z, :].OP
            plt.scatter(x, y, label="B+19", s=1)       
        
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )
        
        plt.legend()
        plt.title(f"M={M}, Z={Z}")
        plt.yscale("log")
        plt.show()

In [ ]:
R18.Z.unique()

In [ ]:
for Z in [0.0001, 0.006, 0.01, 0.02]:
    plt.figure(figsize=(4, 2))

    for M in [1., 1.65, 2., 3., 4., 5., 7.]:
        x  = atomic_numer(R18_indexed.loc[M, Z, :].index.values)
        y = R18_indexed.loc[M, Z, :].OP

        idx = np.argsort(x)
        plt.plot(x[idx], y[idx], label=M, marker="o", markersize=1)
      
        
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )

        if np.max(y) > 10:
            plt.yscale("log")

        plt.legend()
        plt.title(f"Z={Z}")
        if M in [3., 7.]:
            plt.show()
            plt.figure(figsize=(4, 2))

            

In [ ]:
for Z in [0.001]:

    for M in [1.65, 2., 3., 5.]:

        x  = atomic_numer(R18_indexed.loc[M, Z, :].index.values)
        y = R18_indexed.loc[M, Z, :].OP
        plt.scatter(x, y, label="R+18", s=3)
    
        if (M, Z) in [(2., 0.001), (3., 0.001), (2., 0.002)]:
            x  = atomic_numer(B21_indexed.loc[M, Z, :].index.values)
            y = B21_indexed.loc[M, Z, :].OP
            plt.scatter(x, y, label="B+19", s=1)       
        
        plt.xlabel("atomic number")
        plt.ylabel("overproduction factor")
        plt.axhline(1, color="black", )

        plt.yscale("log")
        plt.legend()
        plt.title(f"M={M}, Z={Z}")
        plt.show()